setup

In [ ]:
import os
from pathlib import Path
input_dir = Path("/content/input")
output_dir = Path("/content/output")
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

!git clone https://github.com/yy981-jp/AI_Whisper-CSVBackup_uishig.git /content/output
%cd content/output

!pip install faster-whisper
from faster_whisper import WhisperModel
model = WhisperModel("large-v3-turbo", device="cuda", compute_type="float16")
# model = WhisperModel("small", device="cuda", compute_type="float16")

def transcribe_audio(mp3_path):
    print(f"Transcribing {mp3_path.name}...")

    segments, info = model.transcribe(
        str(mp3_path),
        language="ja",
        task="transcribe"
    )

    out_csv = output_dir / (mp3_path.stem + ".csv")

    total_duration = info.duration

    with open(out_csv, "w", encoding="utf-8") as f:
        for seg in segments:
            ms = int(seg.start * 1000)
            f.write(f"{ms},{seg.text.strip()}\n")
            progress = (seg.end / total_duration) * 100
            print(f"{mp3_path.name}: {progress:.2f}%")

    print(f"Finished transcribing {mp3_path.name}. Output saved to {out_csv}")


proc (multi)

In [ ]:
mp3_files = list(input_dir.glob("*.opus"))

for mp3Input in mp3_files:
    transcribe_audio(mp3Input)
    !git add .
    !git commit -m "update"
    !git push